## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [ ]:
!pip install pymorphy2

In [ ]:
import pandas as pd
import numpy as np
import math
import nltk 
import string

In [ ]:
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from pymorphy2 import MorphAnalyzer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Data/women-clothing-accessories.csv", sep="\t", usecols=[0, 1])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)

In [ ]:
new_data = data

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
def lemmatization(data):
  morph_analyzer = MorphAnalyzer()
  sentences = []
  for sentence in data:
    for symbol in string.punctuation:
      sentence = sentence.replace(symbol, " ")
    processed_sentences = [morph_analyzer.parse(word)[0].normal_form.lower() for word in word_tokenize(sentence)]
    sentences.append(" ".join(processed_sentences))
  return sentences

In [ ]:
new_data.review = lemmatization(data.review)

In [ ]:
my_x_train, my_x_test, my_y_train, my_y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)

In [ ]:
vectorize_result = []

In [ ]:
def fit_transform(vectorizer):
  return vectorizer, vectorizer.fit_transform(my_x_train)

def generator(from_min_n, to_min_n, to_max_n):
  ngram_parameters = []
  for min_n in range(from_min_n, to_min_n):
    for max_n in range(min_n, to_max_n):
      ngram_parameters.append((min_n, max_n))
  return list(ngram_parameters)

def naive_bayes(vectorizer, vectorizer_x_train):
  my_clf = MultinomialNB()
  my_clf.fit(vectorizer_x_train, my_y_train)  
  vectorizer_x_test = vectorizer.transform(my_x_test)
  my_pred = my_clf.predict(vectorizer_x_test)
  print("ngram_range:", str(vectorizer.ngram_range), "analyzer:", str(vectorizer.analyzer))
  print(classification_report(my_y_test, my_pred, output_dict=False))
  return classification_report(my_y_test, my_pred, output_dict=True), vectorizer

###CountVectorizer word analyzer

In [ ]:
for ngram_range in generator(2, 6, 6):
  count_vectorizer, count_vectorizer_x_train = fit_transform(CountVectorizer(ngram_range=ngram_range, stop_words=stopwords.words("russian"), analyzer="word"))
  vectorize_result.append(naive_bayes(count_vectorizer, count_vectorizer_x_train))

ngram_range: (2, 2) analyzer: word
              precision    recall  f1-score   support

    neautral       0.59      0.54      0.56      8980
    negative       0.71      0.65      0.68      9126
    positive       0.74      0.86      0.80      8895

    accuracy                           0.68     27001
   macro avg       0.68      0.69      0.68     27001
weighted avg       0.68      0.68      0.68     27001

ngram_range: (2, 3) analyzer: word
              precision    recall  f1-score   support

    neautral       0.59      0.52      0.55      8980
    negative       0.70      0.66      0.68      9126
    positive       0.73      0.87      0.79      8895

    accuracy                           0.68     27001
   macro avg       0.67      0.68      0.68     27001
weighted avg       0.67      0.68      0.67     27001

ngram_range: (2, 4) analyzer: word
              precision    recall  f1-score   support

    neautral       0.59      0.51      0.55      8980
    negative       0.70 

###TfidfVectorizer

In [ ]:
for ngram_range in generator(2, 4, 4):
  for max_df in [0.1, 0.5]:
    for min_df in [0, 0.01]:
      for max_features in [2048, 8192, 32768]:
        print(max_df, min_df)
        tfidf_vectorizer, tfidf_vectorizer_x_train = fit_transform(TfidfVectorizer(ngram_range=ngram_range, max_df=max_df, min_df=min_df, max_features=max_features))
        vectorize_result.append(naive_bayes(tfidf_vectorizer, tfidf_vectorizer_x_train))

0.1 0
ngram_range: (2, 2) analyzer: word
              precision    recall  f1-score   support

    neautral       0.57      0.61      0.59      8980
    negative       0.74      0.59      0.66      9126
    positive       0.74      0.84      0.79      8895

    accuracy                           0.68     27001
   macro avg       0.68      0.68      0.68     27001
weighted avg       0.68      0.68      0.68     27001

0.1 0
ngram_range: (2, 2) analyzer: word
              precision    recall  f1-score   support

    neautral       0.59      0.64      0.62      8980
    negative       0.74      0.62      0.68      9126
    positive       0.80      0.85      0.82      8895

    accuracy                           0.70     27001
   macro avg       0.71      0.71      0.70     27001
weighted avg       0.71      0.70      0.70     27001

0.1 0
ngram_range: (2, 2) analyzer: word
              precision    recall  f1-score   support

    neautral       0.60      0.65      0.62      8980
    ne

###CountVectorizer char analyzer

In [ ]:
for ngram_range in generator(2, 6, 6):
  count_vectorizer, count_vectorizer_x_train = fit_transform(CountVectorizer(ngram_range=ngram_range, stop_words=stopwords.words("russian"), analyzer="char"))
  vectorize_result.append(naive_bayes(count_vectorizer, count_vectorizer_x_train))

ngram_range: (2, 2) analyzer: char
              precision    recall  f1-score   support

    neautral       0.53      0.62      0.57      8980
    negative       0.68      0.57      0.62      9126
    positive       0.75      0.74      0.75      8895

    accuracy                           0.64     27001
   macro avg       0.65      0.64      0.65     27001
weighted avg       0.65      0.64      0.65     27001

ngram_range: (2, 3) analyzer: char
              precision    recall  f1-score   support

    neautral       0.56      0.67      0.61      8980
    negative       0.71      0.59      0.64      9126
    positive       0.82      0.79      0.80      8895

    accuracy                           0.68     27001
   macro avg       0.69      0.68      0.68     27001
weighted avg       0.69      0.68      0.68     27001

ngram_range: (2, 4) analyzer: char
              precision    recall  f1-score   support

    neautral       0.57      0.68      0.62      8980
    negative       0.72 

###Вывод

In [ ]:
pattern = re.compile(r"[a-zA-Z]+Vectorizer|'word'|'char'|\([0-9,]+,[ 0-9]+\)|min_df=[0-9.]|max_df=[0-9.]+|features=[0-9]+")

In [ ]:
raw_data = []
for vectorize in vectorize_result:
  score = vectorize[0]["weighted avg"]
  names_with_params = pattern.findall(str(vectorize[1]))
  raw_data.append({"Vectorizer": names_with_params[0], "Analyzer": names_with_params[1], "Parameters": names_with_params[2:], "Precision": score["precision"], "Recall": score["recall"], "F1-Score": score['f1-score']})
data_result = pd.DataFrame(raw_data)
data_result.sort_values(by=["Precision"], ascending=False).head(10)

,Vectorizer,Analyzer,Parameters,Precision,Recall,F1-Score
55,CountVectorizer,'char',"[max_df=1.0, min_df=1, (5, 5)]",0.723708,0.713122,0.714821
54,CountVectorizer,'char',"[max_df=1.0, min_df=1, (4, 5)]",0.719849,0.709122,0.710857
18,TfidfVectorizer,'word',"[max_df=0.5, features=32768, min_df=0, (2, 2)]",0.717944,0.714418,0.714325
12,TfidfVectorizer,'word',"[max_df=0.1, features=32768, min_df=0, (2, 2)]",0.717944,0.714418,0.714325
52,CountVectorizer,'char',"[max_df=1.0, min_df=1, (3, 5)]",0.717305,0.705937,0.707838
24,TfidfVectorizer,'word',"[max_df=0.1, features=32768, min_df=0, (2, 3)]",0.716079,0.712233,0.711735
30,TfidfVectorizer,'word',"[max_df=0.5, features=32768, min_df=0, (2, 3)]",0.716079,0.712233,0.711735
49,CountVectorizer,'char',"[max_df=1.0, min_df=1, (2, 5)]",0.715904,0.704122,0.706116
53,CountVectorizer,'char',"[max_df=1.0, min_df=1, (4, 4)]",0.714869,0.704011,0.705795
51,CountVectorizer,'char',"[max_df=1.0, min_df=1, (3, 4)]",0.712026,0.700604,0.702500


Самым лучшим векторайзером стал **CountVectorizer с char** анализатором с гиперпараметром *ngram=(5, 5)*. Также неплохие показатели показывает **TfidfVectorizer** с параметрами *max_df=0.5* *min_df=0* *features=32768* *ngram=(2, 2)*. **CountVectorizer с word** анализатором в первую десятку не попал(

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
text = "Уважаемые коллеги, добрый день!"
re.findall(r"\b\w{2}", text)

['Ув', 'ко', 'до', 'де']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = "Уважаемые студенты! Мы на первом занятии специально создали тему, чтобы вы в неё загружали свои работы в отдельные followup discussions."\
       "Не надо создавать новые темы типа «моя работа» или «лаба 1»."\
       "Вот вы представьте, мне сейчас 270 человек создадут тут каждый по такой теме под каждую из своих работ."\
       "Форум тут же превратиться в помойку, чтобы этого избежать, все такие темы будут удалятся мною."\
       "При повторных нарушениях буду снижать баллы за низкий уровень цифровой культуры и неспособность к грамотной комммуникации в Интернете."

re.split(r"\.", text, maxsplit=2)

['Уважаемые студенты! Мы на первом занятии специально создали тему, чтобы вы в неё загружали свои работы в отдельные followup discussions',
 'Не надо создавать новые темы типа «моя работа» или «лаба 1»',
 'Вот вы представьте, мне сейчас 270 человек создадут тут каждый по такой теме под каждую из своих работ.Форум тут же превратиться в помойку, чтобы этого избежать, все такие темы будут удалятся мною.При повторных нарушениях буду снижать баллы за низкий уровень цифровой культуры и неспособность к грамотной комммуникации в Интернете.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text = "284189 Папикян Сергей Седракович M33011"
re.sub(r"\d", "DIG", text)

'DIGDIGDIGDIGDIGDIG Папикян Сергей Седракович MDIGDIGDIGDIGDIG'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
text = "Мой ИСУ: https://isu.ifmo.ru/pls/apex/f?p=2437:7:108188039133937:::::, Cайт Факультета: https://fitp.itmo.ru/, ПСЖ: https://neerc.ifmo.ru/lgd.pdf"
re.sub(r"http[s]?://[\w.:?/=]+", "", text)

'Мой ИСУ: , Cайт Факультета: , ПСЖ: '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
text = "Выбирай ИТМО - и не выбирай вообще"
pattern = re.compile(r"\w{4,}")
pattern.findall(text)

['Выбирай', 'ИТМО', 'выбирай', 'вообще']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
text = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
pattern = re.compile(r"@[\w.]+")
pattern.findall(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']